In [51]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [52]:
# ------------------------------------------------------------
# Helper function 1: w_function
# Produce coefficient list of w_{n+1}(x) = Π (x - x_i)
# ------------------------------------------------------------
def w_function(x_points):
    coeff = np.array([1.0])  # start with polynomial 1
    for xi in x_points:
        coeff = np.convolve(coeff, np.array([1.0, -xi]))  # multiply by (x - xi)
    return coeff  # highest to lowest degree

In [53]:
# ------------------------------------------------------------
# Helper function 2: c_function
# Produce coefficient list of C_i(x) = w_{n+1}(x) / (x - x_i)
# ------------------------------------------------------------
def c_function(w_coeffs, xi):
    n = len(w_coeffs)
    c_coeffs = np.zeros(n - 1)
    remainder = 0.0
    # Synthetic division by (x - xi)
    for j in range(n - 1):
        if j == 0:
            c_coeffs[j] = w_coeffs[j]
        else:
            c_coeffs[j] = w_coeffs[j] + xi * c_coeffs[j - 1]
    remainder = w_coeffs[-1] + xi * c_coeffs[-1]
    # We expect remainder ≈ 0 if everything is correct
    return c_coeffs


# Lagrange Interpolation

## Algorithm

- Algorithm: Lagrange Interpolation

**Given:** $(x_0, y_0), (x_1, y_1), \dots, (x_n, y_n)$  
**Goal:** Find coefficients of polynomial $L(x)$ passing through all given points.

- Steps

1. Compute the base polynomial $w_{n+1}(x) = \prod_{i=0}^{n} (x - x_i)$  
   → obtain coefficient list of $w_{n+1}(x)$.

2. For each $i = 0, 1, \dots, n$:
   - Compute $D_i = \dfrac{y_i}{\prod_{k=0, k \ne i}^{n} (x_i - x_k)}$
   - Compute $C_i(x) = \dfrac{w_{n+1}(x)}{(x - x_i)}$
   - Form $L_i(x) = D_i \cdot C_i(x)$

3. Sum all sub-polynomials:  
   $L(x) = \sum_{i=0}^{n} L_i(x)$

4. Return the coefficient list of $L(x)$ from lowest to highest degree.

**Condition:** All $x_i$ values must be distinct.  
**Output:** Coefficients of $L(x)$.


In [54]:

# ------------------------------------------------------------
# Main function: lagrange_interpolation
# ------------------------------------------------------------
def lagrange_interpolation(points):
    x_points = np.array([p[0] for p in points], dtype=float)
    y_points = np.array([p[1] for p in points], dtype=float)
    n = len(points) - 1

    # Step 1: Compute w(x)
    w_coeffs = w_function(x_points)

    w_coeffs_df = pd.DataFrame({'Degree': list(range(len(w_coeffs)-1, -1, -1)), 'Coeff': w_coeffs})

    # Step 2: Compute each L_i(x)
    all_Li = []
    table_data = []

    for i in range(n + 1):
        xi, yi = x_points[i], y_points[i]
        
        # D_i calculation
        denom = np.prod([xi - xk for j, xk in enumerate(x_points) if j != i])
        D_i = yi / denom
        
        # C_i(x)
        C_i = c_function(w_coeffs, xi)
        
        # L_i(x) = D_i * C_i(x)
        L_i = D_i * C_i
        all_Li.append(L_i)
        
        # Store intermediate results
        table_data.append({
            'i': i,
            'x_i': xi,
            'y_i': yi,
            'denom': denom,
            'D_i': D_i,
            'C_i(x) coeffs': C_i.tolist(),
            'L_i(x) coeffs': L_i.tolist()
        })

    sub_lagrange_df = pd.DataFrame(table_data)

    # Step 3: Sum all L_i(x)
    max_len = max(len(Li) for Li in all_Li)
    L_coeffs = np.zeros(max_len)
    for Li in all_Li:
        L_coeffs[-len(Li):] += Li  # align degrees

    coeff_table = pd.DataFrame({
        'Degree': list(range(len(L_coeffs))),
        'Coeff': L_coeffs,
    })

    return w_coeffs_df, sub_lagrange_df, L_coeffs

## Result

In [55]:
# ------------------------------------------------------------
# Example run
# ------------------------------------------------------------
points = [(1, 17), (2, 17.5), (3, 76), (4, 210.5), (7, 1970)]
w_coeffs_df, sub_lagrange_df, L_coeffs = lagrange_interpolation(points)

In [56]:
w_coeffs_df

,Degree,Coeff
0,5,1.0
1,4,-17.0
2,3,105.0
3,2,-295.0
4,1,374.0
5,0,-168.0


In [57]:
sub_lagrange_df

,i,x_i,y_i,denom,D_i,C_i(x) coeffs,L_i(x) coeffs
0,0,1.0,17.0,36.0,0.472222222222,"[1.0, -16.0, 89.0, -206.0, 168.0]","[0.4722222222222222, -7.555555555555555, 42.02..."
1,1,2.0,17.5,-10.0,-1.750000000000,"[1.0, -15.0, 75.0, -145.0, 84.0]","[-1.75, 26.25, -131.25, 253.75, -147.0]"
2,2,3.0,76.0,8.0,9.500000000000,"[1.0, -14.0, 63.0, -106.0, 56.0]","[9.5, -133.0, 598.5, -1007.0, 532.0]"
3,3,4.0,210.5,-18.0,-11.694444444444,"[1.0, -13.0, 53.0, -83.0, 42.0]","[-11.694444444444445, 152.02777777777777, -619..."
4,4,7.0,1970.0,360.0,5.472222222222,"[1.0, -10.0, 35.0, -50.0, 24.0]","[5.472222222222222, -54.72222222222222, 191.52..."


In [58]:
L_coeffs

array([   2. ,  -17. ,   81. , -153.5,  104.5])